In [173]:
import sys
import datetime
import pandas as pd
import numpy as np
import datetime as dt
import time
import math
import matplotlib.pyplot as plt
import random
from pandas.tseries.offsets import DateOffset
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import plotly.graph_objects as go
from scipy.signal import argrelextrema
from scipy.stats import linregress
#import autosklearn

sys.path.append('C:/workarea/mylib/')
sys.path.append('C:/workarea/mylib/quant')
sys.path.append('C:/workarea/mylib/history')
import quant as q
import quant.helpers as hlp
from history import bt, history_tools as ht
asys = q.AnalysisComponents()

import warnings
warnings.filterwarnings("ignore")
locx = 'S:/Matrix/IA/IADE/0_Personal/1_Antonio/risk_return_contribution/pricing/files/'

def get_freq(data, datecol='t', freq='5min'):
    data = data.set_index(pd.DatetimeIndex(data[datecol]))
    data.index.name = None
    df = pd.DataFrame()
    df['o'] = data.o.resample(freq).first()
    df['h'] = data.h.resample(freq).max()
    df['l'] = data.l.resample(freq).min()
    df['c'] = data.c.resample(freq).last()
    df['v'] = data.v.resample(freq).sum()
    return df.reset_index().rename(columns={'index':datecol})

In [5]:
## get pricing, convert to different frequencies
df5 = pd.read_csv(locx + 'BU5m.txt')
df15 = get_freq(df5, freq='15min')
df30 = get_freq(df5, freq='30min')
df1h = get_freq(df15, freq='1h')
df1d = get_freq(df1h, freq='1d')
df4h = get_freq(df1h, freq='4h')

In [179]:
def put_order(pr, order_type, trade, quantity=0.003):
    message = 'set', trade, order_type, 'order of', str(quantity),'@', str(pr)
    order_id = random.randint(0,10000)
    print(message, ' - order_id:', order_id)
    return order_id

def cancel_order(order_id):
    print('cancel order - ', order_id)
    return 'OK'

def mean_rev_xper_old(conct_target, pr, prev_pr, prev_sign, prev_conct, thresnull=0.00014, price_buffer=1/10000):
    
    ret = (pr - prev_pr) / prev_pr
    ret = 0 if abs(ret) < thresnull else ret
    sign = 'pos' if ret > 0 else 'neg' if ret < 0 else ''
    
    if sign==prev_sign:
        conct = prev_conct+1
        if conct == conct_target:
            #trade='L' if sign=='neg' else 'S' if sign=='pos' else ''
            
            if sign=='neg':
                trade='L'
                order_id = put_order(round(pr - pr*price_buffer, 2), order_type='limit', trade=trade, quantity=0.003)
            elif sign=='pos':
                trade='S'
                order_id = put_order(round(pr + pr*price_buffer, 2), order_type='limit', trade=trade, quantity=0.003)
            else:
                trade=''           
        else:
            trade = ''
    else:
        conct = 1
        trade = ''
        
    return trade, ret, pr, sign, conct

def mean_rev_xper(conct_target, pr, prev_pr, prev_sign, prev_conct, thresnull=0.00014, price_buffer=1/10000):
    
    trade=''
    order_id = 0
    conct = 1
    ret = (pr - prev_pr) / prev_pr
    ret = 0 if abs(ret) < thresnull else ret
    sign = 'pos' if ret > 0 else 'neg' if ret < 0 else ''
    
    if sign==prev_sign:
        conct = prev_conct+1
        if conct == conct_target:
            
            if sign=='neg':
                trade='L'
                order_id = put_order(round(pr - pr*price_buffer, 2), order_type='limit', trade=trade, quantity=0.003)
            elif sign=='pos':
                trade='S'
                order_id = put_order(round(pr + pr*price_buffer, 2), order_type='limit', trade=trade, quantity=0.003)    
        
    return trade, ret, pr, sign, conct, order_id

In [187]:
dfx = df15.iloc[120:142,:].reset_index(drop=True)

dfx = df15.tail(30).reset_index(drop=True)

dfx = df15[146293-50:146296].reset_index(drop=True)

thresnull = 0.00015
conct_target = 3
periods_in_trade = 2
secondsbuffer = 1
price_buffer=1/10000

prev_pr, prev_sign, prev_conct, trade = 4289, np.nan, 0, ''

dfr = pd.DataFrame()
ct = 0
ct_trade = 0
in_trade = 0

position=True

while ct < len(dfx):
    
    #print(dfx.loc[ct].t)
    
    if ct_trade == periods_in_trade:
        #check if position
        if not position:
            print('cancel order', last_order_id)
            in_trade = 0
        if position:
            print('close trade')
            
            close_pr = dfx.loc[ct].c
            
            if current_trade=='L':
                order_id = put_order(round(close_pr + close_pr*price_buffer, 2), order_type='limit', trade='S', quantity=0.003)
            elif current_trade=='S':
                order_id = put_order(round(close_pr - close_pr*price_buffer, 2), order_type='limit', trade='L', quantity=0.003)   
            
            in_trade = 0
    
    trade, ret, prev_pr, prev_sign, prev_conct, order_id = mean_rev_xper(conct_target, dfx.loc[ct].c, prev_pr, prev_sign, prev_conct, thresnull=thresnull)
    #print(trade, ret, row.c, prev_pr, prev_sign, prev_conct)    
    if trade:
        in_trade = 1
        last_order_id = order_id
        current_trade = trade
    
    
    dfr_ = pd.DataFrame([dt.datetime.now(), prev_pr, ret, prev_sign, prev_conct, trade]).T
    dfr_.columns = ['t','price','ret','sign','conct','trade']
    dfr = pd.concat([dfr, dfr_], axis=0)
    
    ct +=1
    ct_trade = (ct_trade+1)*in_trade
    

('set', 'S', 'limit', 'order of', '0.003', '@', '61841.41')  - order_id: 2595
close trade
('set', 'L', 'limit', 'order of', '0.003', '@', '62002.57')  - order_id: 5837
('set', 'L', 'limit', 'order of', '0.003', '@', '61814.59')  - order_id: 2212
close trade
('set', 'S', 'limit', 'order of', '0.003', '@', '61949.95')  - order_id: 877
('set', 'L', 'limit', 'order of', '0.003', '@', '61262.48')  - order_id: 9222
close trade
('set', 'S', 'limit', 'order of', '0.003', '@', '61339.91')  - order_id: 9148
('set', 'S', 'limit', 'order of', '0.003', '@', '61537.37')  - order_id: 1773
close trade
('set', 'L', 'limit', 'order of', '0.003', '@', '61287.82')  - order_id: 3846
('set', 'S', 'limit', 'order of', '0.003', '@', '61801.09')  - order_id: 2038
close trade
('set', 'L', 'limit', 'order of', '0.003', '@', '61631.57')  - order_id: 1919
('set', 'L', 'limit', 'order of', '0.003', '@', '61777.65')  - order_id: 4002
close trade
('set', 'S', 'limit', 'order of', '0.003', '@', '61827.27')  - order_id

In [181]:
dfr

,t,price,ret,sign,conct,trade
0,2022-01-19 17:02:12.353127,60617,13.1331,pos,1,
0,2022-01-19 17:02:12.355126,60350,-0.00440487,neg,1,
0,2022-01-19 17:02:12.356126,61290.1,0.015578,pos,1,
0,2022-01-19 17:02:12.359128,61352.9,0.00102447,pos,2,
0,2022-01-19 17:02:12.361126,61835.2,0.00786124,pos,3,S
0,2022-01-19 17:02:12.363126,62019.1,0.00297355,pos,4,
0,2022-01-19 17:02:12.365125,62008.8,-0.000166562,neg,1,
0,2022-01-19 17:02:12.366154,61760.5,-0.00400427,neg,2,
0,2022-01-19 17:02:12.368139,62092.4,0.00537447,pos,1,
0,2022-01-19 17:02:12.369126,62000,-0.00148859,neg,1,
